In [1]:
import pandas as pd
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
import os
from datetime import datetime
import pdb;
from grocery_ml_tensorflow import GroceryML
from grocery_ml_core import GroceryMLCore
from hidden_layer_param_builder import HiddenLayerParamSetBuilder

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: f"{x:.6f}")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)

print(os.getcwd())
# print("GPUs Available:", tf.config.list_physical_devices('GPU'))
#tf.debugging.set_log_device_placement(True)

# def run_all_experiments(df, model_param_sets, output_dir):
#     total = len(model_param_sets)
#     print(f"run_all_experiments() when: {datetime.now()}  output_dir: {output_dir}");
#     for index, params in enumerate(model_param_sets, 1):
#         print(f"Running Exp {index}/{total}...")
#         groceryML.run_experiment(df,  params["buildParams"], params["trainParams"], output_dir)

def run_all_experiments(df, model_param_sets, output_dir, start_date=pd.Timestamp.now(), days=1):
    total = len(model_param_sets)
    print(f"run_all_experiments_conseq() when: {datetime.now()} output_dir: {output_dir}")
    for index, params in enumerate(model_param_sets, 1):
        print(f"Running Exp {index}/{total}...")
        groceryML.run_experiment_with_consecutive_predictions(
            df,
            params["buildParams"],
            params["trainParams"],
            output_dir,
            start_date,
            days
        )

try:
    groceryML = GroceryML();
    groceryMLCore = GroceryMLCore();
    groceryML.build_training_df()
    if groceryML.training_df is None:
        raise();
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}.csv");
except Exception as ex: 
    print(ex)
    ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    groceryML.training_df.to_csv(f"training_df-{ts}-exception.csv");


C:\Users\steve\source\repos\grocery-ml
Building Training DF: Start
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55992 entries, 0 to 55991
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           55992 non-null  datetime64[ns]
 1   itemId         55992 non-null  int64         
 2   source         55992 non-null  object        
 3   item           55992 non-null  object        
 4   qty            1417 non-null   float64       
 5   didBuy_target  55992 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(2)
memory usage: 2.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208881 entries, 0 to 208880
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date    

In [2]:

# quick test shot
layers = [
    { "units": 1, "activation": "relu" }
]
modelParamsList = []
modelParamsList.append({
    "trainParams": { "epochs": 1 },
    "buildParams": {
    "embedding_dim": 1,
    "layers": layers,
    "activation": "relu",
    "output_activation": "sigmoid",
    "optimizer": "adam",
    "learning_rate": 0.01,
    "loss": "binary_crossentropy",
    "metrics": "BinaryCrossentropy"
    }
})
ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
run_all_experiments(groceryML.training_df, modelParamsList, f"exp/keras/keras-1/test-shot-no-trust-{ts}")


run_all_experiments_conseq() when: 2026-01-19 21:50:58.191292 output_dir: exp/keras/keras-1/test-shot-no-trust-2026_01_19_21_50_58
Running Exp 1/1...
Creating dir: exp/keras/keras-1/test-shot-no-trust-2026_01_19_21_50_58\e1_l1_ep1_sig_442
run_experiment_with_consecutive_predictions() exp_dir: exp/keras/keras-1/test-shot-no-trust-2026_01_19_21_50_58\e1_l1_ep1_sig_442
when: 2026-01-19 21:50:58.191762 params: {'epochs': 1}
normalize_features()
train_model()
build_prediction_input() prediction_date=2026-01-19 21:49:47.357831
Building Live DF: Start
_build_combined_df()
_build_sources()
creating target col: didBuy_target
insert_negative_samples()


C:\Users\steve\source\repos\grocery-ml\grocery_ml_core.py:344: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  additional_rcpts_df["date"] = pd.to_datetime(additional_rcpts_df["date"])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57146 entries, 0 to 57145
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   date           57146 non-null  datetime64[ns]
 1   itemId         57146 non-null  int64         
 2   source         57146 non-null  object        
 3   item           57146 non-null  object        
 4   qty            1448 non-null   float64       
 5   didBuy_target  57146 non-null  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(1), object(2)
memory usage: 2.4+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210709 entries, 0 to 210708
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   date           210709 non-null  datetime64[ns]
 1   source         210709 non-null  object        
 2   itemId         210709 non-null  int64         
 3   item           210

InvalidArgumentError: Graph execution error:

Detected at node 'model/item_embedding/embedding_lookup' defined at (most recent call last):
    File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "C:\ProgramData\miniconda3\envs\grocery-ml-keras\lib\site-packages\tornado\platform\asyncio.py", line 211, in star

In [ ]:
###########################################################

modelParamsList = []
layers = [
    { "units": 64, "activation": "relu" },
    { "units": 64, "activation": "relu" },
    { "units": 64, "activation": "relu" },
    { "units": 64, "activation": "relu" },
    { "units": 64, "activation": "relu" },
    { "units": 64, "activation": "relu" },
]

output_activations = ["sigmoid"]
embDims = [100]
ep = [ 280 ]
lr = [.0001,]
metrics = ["Accuracy", "MAE", "MSE", "MAPE", "MSLE", "Precision", "Recall", "AUC",  "BinaryCrossentropy", "RootMeanSquaredError"]

for dim in embDims:
    for act in output_activations:
        for epoch in ep: 
            for l in lr:
                modelParamsList.append({
                    "trainParams": { "epochs": epoch },
                    "buildParams": {
                        "embedding_dim": dim,
                        "layers": layers,
                        "output_activation": act,
                        "optimizer": "adam",
                        "learning_rate": l,
                        "loss": "binary_crossentropy" if act == "sigmoid" else "mse",
                        "metrics": metrics
                    }
        })


ts = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
exp_dir = rf"F:\exp\keras\optuna\study-no-itemPurchaseCount-20260115_003731\trial-289\control"
path = os.path.join(exp_dir, ts)
start_date = pd.Timestamp("2026-01-15 17:20:26")
run_all_experiments( groceryML.training_df, modelParamsList, path, start_date=start_date, days = 5)

